# Warp Shell - Comoving Frame

This notebook demonstrates the **warp shell metric** in a comoving reference frame. The warp shell is a spherical shell of matter with specific density and pressure profiles that can create a warp effect for objects within the shell.

## What is a Warp Shell?

The warp shell metric describes a spherical shell of matter that generates a spacetime curvature capable of producing warp effects. Key features:

- **Spherical geometry**: Matter is distributed in a shell between inner radius R1 and outer radius R2
- **TOV equations**: The Tolman-Oppenheimer-Volkoff equations govern the pressure and density profiles within the shell
- **Comoving frame**: The metric is constructed in the frame moving with the warp drive
- **Controlled warp effect**: Objects inside the shell can experience controlled spacetime warping

Reference: [IOP Science Article on Constant Velocity Warp Shells](https://iopscience.iop.org/article/10.1088/1361-6382/ad26aa)

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Import WarpFactory modules
from warpfactory.metrics.warp_shell import get_warp_shell_comoving_metric
from warpfactory.analyzer.eval_metric import eval_metric
from warpfactory.units.constants import c, G

# Set plotting parameters
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12

## Metric Creation

First, we define the parameters for our warp shell and create the metric. The key parameters are:

- **R1**: Inner radius of the shell (meters)
- **R2**: Outer radius of the shell (meters)
- **m**: Total mass of the shell (kg)
- **v_warp**: Velocity of the warp drive as a fraction of c
- **sigma**: Sharpness parameter for the shell boundaries
- **smooth_factor**: Factor controlling smoothing of density/pressure profiles
- **do_warp**: Whether to enable the warp effect inside the shell

In [ ]:
# Define shell geometry parameters
R1 = 10.0  # Inner radius (meters)
R2 = 20.0  # Outer radius (meters)
Rbuff = 0.0  # Buffer distance from shell walls

# Calculate grid parameters
space_scale = 5  # Grid resolution scaling
time_scale = 1
cartoon_thickness = 5  # Thickness in z-direction (for computational efficiency)
centered = True  # Center the shell in the grid

# Set up grid size
if centered:
    grid_size = [
        1,
        int(2 * (R2 + 10) * space_scale),
        int(2 * (R2 + 10) * space_scale),
        cartoon_thickness
    ]
else:
    grid_size = [
        1,
        int((R2 + 10) * space_scale),
        int((R2 + 10) * space_scale),
        cartoon_thickness
    ]

print(f"Grid size: {grid_size}")
print(f"Grid dimensions: [time={grid_size[0]}, x={grid_size[1]}, y={grid_size[2]}, z={grid_size[3]}]")

In [ ]:
# Calculate mass and warp parameters
factor = 1.0 / 3.0  # Mass factor (determines compactness of the shell)
c_value = c()  # Speed of light
G_value = G()  # Gravitational constant

# Total mass of the shell (derived from compactness parameter)
m = R2 / (2 * G_value) * c_value**2 * factor

# Warp drive velocity (in units of c)
v_warp = 0.02  # 2% of light speed

# Smoothness and transition parameters
sigma = 0.0  # Sharpness of shell boundaries
smooth_factor = 4000.0  # Smoothing strength for density/pressure profiles
do_warp = True  # Enable warp effect

print(f"\nWarp Shell Parameters:")
print(f"  Inner radius (R1): {R1} m")
print(f"  Outer radius (R2): {R2} m")
print(f"  Shell thickness: {R2 - R1} m")
print(f"  Total mass: {m:.3e} kg")
print(f"  Warp velocity: {v_warp}c = {v_warp * c_value:.3e} m/s")
print(f"  Smooth factor: {smooth_factor}")
print(f"  Warp enabled: {do_warp}")

In [ ]:
# Set up grid scaling and world center
grid_scaling = [
    1.0 / (1000 * c_value),  # Time scaling
    1.0 / space_scale,  # X scaling
    1.0 / space_scale,  # Y scaling
    1.0 / space_scale   # Z scaling
]

if centered:
    world_center = [
        (grid_size[0] + 1) / 2 * grid_scaling[0],
        (grid_size[1] + 1) / 2 * grid_scaling[1],
        (grid_size[2] + 1) / 2 * grid_scaling[2],
        (grid_size[3] + 1) / 2 * grid_scaling[3]
    ]
else:
    world_center = [
        (grid_size[0] + 1) / 2 * grid_scaling[0],
        5.0,
        5.0,
        (grid_size[3] + 1) / 2 * grid_scaling[3]
    ]

print(f"\nGrid Configuration:")
print(f"  Grid scaling: {grid_scaling}")
print(f"  World center: {world_center}")

In [ ]:
# Create coordinate arrays for plotting
if centered:
    x = np.linspace(0, 2 * (R2 + 10), grid_size[1] - 4)
    y = np.linspace(0, 2 * (R2 + 10), grid_size[2] - 4)
    xlimit = [0, 2 * (R2 + 10)]
else:
    x = np.linspace(0, (R2 + 10), grid_size[1] - 4)
    y = np.linspace(0, (R2 + 10), grid_size[2] - 4)
    xlimit = [0, (R2 + 10)]

X, Y = np.meshgrid(x, y)
print(f"Coordinate arrays created: X shape = {X.shape}, Y shape = {Y.shape}")

In [ ]:
# Create the warp shell metric
print("Creating warp shell metric...")
print("This may take a few minutes due to the TOV equation integration...\n")

metric_warp_shell = get_warp_shell_comoving_metric(
    grid_size=grid_size,
    world_center=world_center,
    m=m,
    R1=R1,
    R2=R2,
    Rbuff=Rbuff,
    sigma=sigma,
    smooth_factor=smooth_factor,
    v_warp=v_warp,
    do_warp=do_warp,
    grid_scaling=grid_scaling
)

print("\nMetric created successfully!")
print(f"  Name: {metric_warp_shell.name}")
print(f"  Type: {metric_warp_shell.type}")
print(f"  Index: {metric_warp_shell.index}")
print(f"  Coordinates: {metric_warp_shell.coords}")
print(f"  Shape: {metric_warp_shell.shape}")

## Understanding TOV Equations

The **Tolman-Oppenheimer-Volkoff (TOV)** equations describe the structure of a spherically symmetric body of matter in general relativity. For our warp shell:

$$\frac{dP}{dr} = -\frac{G(\rho + P/c^2)(m(r) + 4\pi r^3 P/c^2)}{r^2(1 - 2Gm(r)/(rc^2))}$$

where:
- $P(r)$ is the pressure profile
- $\rho(r)$ is the density profile
- $m(r) = \int_0^r 4\pi r'^2 \rho(r') dr'$ is the mass within radius r

The metric stores these computed profiles in its parameters:

In [ ]:
# Extract and visualize the shell's physical profiles
r_vec = metric_warp_shell.params['rVec']
rho = metric_warp_shell.params['rho']
rho_smooth = metric_warp_shell.params['rhoSmooth']
P = metric_warp_shell.params['P']
P_smooth = metric_warp_shell.params['PSmooth']
M = metric_warp_shell.params['M']
A = metric_warp_shell.params['A']
B = metric_warp_shell.params['B']

# Create figure with physical profiles
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Warp Shell Physical Profiles', fontsize=16, fontweight='bold')

# Density profile
ax = axes[0, 0]
ax.plot(r_vec, rho, 'b-', alpha=0.3, label='Raw density')
ax.plot(r_vec, rho_smooth, 'b-', linewidth=2, label='Smoothed density')
ax.axvline(R1, color='r', linestyle='--', alpha=0.5, label='R1 (inner)')
ax.axvline(R2, color='r', linestyle=':', alpha=0.5, label='R2 (outer)')
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('Density $\\rho$ (kg/m³)')
ax.set_title('Density Profile')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])

# Pressure profile
ax = axes[0, 1]
ax.plot(r_vec, P, 'g-', alpha=0.3, label='Raw pressure')
ax.plot(r_vec, P_smooth, 'g-', linewidth=2, label='Smoothed pressure')
ax.axvline(R1, color='r', linestyle='--', alpha=0.5)
ax.axvline(R2, color='r', linestyle=':', alpha=0.5)
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('Pressure P (Pa)')
ax.set_title('Pressure Profile (from TOV)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])

# Mass profile
ax = axes[0, 2]
ax.plot(r_vec, M, 'purple', linewidth=2)
ax.axvline(R1, color='r', linestyle='--', alpha=0.5)
ax.axvline(R2, color='r', linestyle=':', alpha=0.5)
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('Mass m(r) (kg)')
ax.set_title('Enclosed Mass Profile')
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])

# Metric function A (temporal component)
ax = axes[1, 0]
ax.plot(r_vec, A, 'orange', linewidth=2)
ax.axvline(R1, color='r', linestyle='--', alpha=0.5)
ax.axvline(R2, color='r', linestyle=':', alpha=0.5)
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('A(r) = $g_{tt}$')
ax.set_title('Metric Function A (Time-Time Component)')
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])

# Metric function B (radial component)
ax = axes[1, 1]
ax.plot(r_vec, B, 'cyan', linewidth=2)
ax.axvline(R1, color='r', linestyle='--', alpha=0.5)
ax.axvline(R2, color='r', linestyle=':', alpha=0.5)
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('B(r) = $g_{rr}$')
ax.set_title('Metric Function B (Radial-Radial Component)')
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])

# Compactness parameter
ax = axes[1, 2]
compactness = 2 * G_value * M / (r_vec * c_value**2)
compactness[0] = 0  # Avoid singularity at r=0
ax.plot(r_vec, compactness, 'brown', linewidth=2)
ax.axvline(R1, color='r', linestyle='--', alpha=0.5)
ax.axvline(R2, color='r', linestyle=':', alpha=0.5)
ax.axhline(1.0, color='k', linestyle='--', alpha=0.3, label='Black hole limit')
ax.set_xlabel('Radius r (m)')
ax.set_ylabel('Compactness $2GM/(rc^2)$')
ax.set_title('Compactness Parameter')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, R2 * 1.5])
ax.set_ylim([0, min(1.2, np.max(compactness[np.isfinite(compactness)]))])

plt.tight_layout()
plt.show()

print("\nPhysical Profile Statistics:")
print(f"  Maximum density: {np.max(rho_smooth):.3e} kg/m³")
print(f"  Maximum pressure: {np.max(P_smooth):.3e} Pa")
print(f"  Total enclosed mass: {M[-1]:.3e} kg")
print(f"  Maximum compactness: {np.max(compactness[np.isfinite(compactness)]):.4f}")

## Visualizing the Metric Components

Now let's visualize the metric tensor components in a 2D slice through the warp shell. The metric tensor in covariant form is:

$$g_{\mu\nu} = \begin{pmatrix}
g_{00} & g_{01} & g_{02} & g_{03} \\
g_{10} & g_{11} & g_{12} & g_{13} \\
g_{20} & g_{21} & g_{22} & g_{23} \\
g_{30} & g_{31} & g_{32} & g_{33}
\end{pmatrix}$$

We'll plot the unique components (taking advantage of symmetry).

In [ ]:
# Helper function for diverging colormap centered at zero
def get_diverging_colormap(data):
    """Create a diverging colormap (red-white-blue) centered at zero"""
    vmax = np.max(np.abs(data[np.isfinite(data)]))
    return 'RdBu_r', -vmax, vmax

# Extract metric components for visualization
z_offset = 0  # Offset from center slice
z_idx = (grid_size[3] + 1) // 2 + z_offset
t_idx = 0  # Time slice

# Select which components to plot (unique components)
components = [
    (0, 0, '$g_{00}$ (Time-Time)'),
    (0, 1, '$g_{01}$ (Time-X)'),
    (0, 2, '$g_{02}$ (Time-Y)'),
    (1, 1, '$g_{11}$ (X-X)'),
    (1, 2, '$g_{12}$ (X-Y)'),
    (2, 2, '$g_{22}$ (Y-Y)')
]

# Create the plot
fig = plt.figure(figsize=(18, 12))
fig.suptitle('Warp Shell Metric Components (2D Slice)', fontsize=16, fontweight='bold')

for idx, (i, j, title) in enumerate(components):
    ax = fig.add_subplot(2, 3, idx + 1)
    
    # Extract the component data (trim boundary points)
    data = metric_warp_shell[(i, j)][t_idx, 2:-2, 2:-2, z_idx].T
    
    # Get colormap and limits
    cmap, vmin, vmax = get_diverging_colormap(data)
    
    # Plot as surface
    im = ax.contourf(X, Y, data, levels=50, cmap=cmap, vmin=vmin, vmax=vmax)
    
    # Add shell boundaries
    circle_R1 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R1, color='red', fill=False, linestyle='--', linewidth=2, label='R1 (inner)'
    )
    circle_R2 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R2, color='red', fill=False, linestyle=':', linewidth=2, label='R2 (outer)'
    )
    ax.add_patch(circle_R1)
    ax.add_patch(circle_R2)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.set_xlim(xlimit)
    ax.set_ylim(xlimit)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    
    if idx == 0:
        ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print("\nMetric Component Values at Origin:")
origin_idx = [t_idx, grid_size[1]//2, grid_size[2]//2, z_idx]
print(f"  g_00 (time-time): {metric_warp_shell[(0,0)][tuple(origin_idx)]:.6f}")
print(f"  g_01 (time-x): {metric_warp_shell[(0,1)][tuple(origin_idx)]:.6f}")
print(f"  g_11 (x-x): {metric_warp_shell[(1,1)][tuple(origin_idx)]:.6f}")
print(f"  g_22 (y-y): {metric_warp_shell[(2,2)][tuple(origin_idx)]:.6f}")
print(f"  g_33 (z-z): {metric_warp_shell[(3,3)][tuple(origin_idx)]:.6f}")

## Energy Tensor and Energy Condition Evaluation

Now we compute the stress-energy tensor and evaluate the energy conditions. This tells us where and how severely the warp shell violates the standard energy conditions of general relativity.

In [ ]:
# Evaluate the metric to get energy tensor and energy conditions
print("Evaluating metric properties...")
print("Computing stress-energy tensor and energy conditions...")
print("This may take several minutes...\n")

results = eval_metric(
    metric_warp_shell,
    try_gpu=False,
    keep_positive=True,
    num_angular_vec=100,
    num_time_vec=10
)

print("\nEvaluation complete!")
print(f"  Energy tensor computed: {results['energy_tensor'].type}")
print(f"  Energy tensor (Eulerian): {results['energy_tensor_eulerian'].type}")
print(f"  Energy conditions computed: Null, Weak, Strong, Dominant")

## Visualizing the Energy Density

The stress-energy tensor $T_{\mu\nu}$ describes the distribution of energy and momentum in spacetime. The $T^{00}$ component represents the energy density as measured by an Eulerian (stationary) observer.

In [ ]:
# Plot energy density
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111)

# Extract energy density (T^00 in Eulerian frame)
energy_density = results['energy_tensor_eulerian'][(0, 0)][t_idx, 2:-2, 2:-2, z_idx].T

# Get colormap
cmap, vmin, vmax = get_diverging_colormap(energy_density)

# Plot
im = ax.contourf(X, Y, energy_density, levels=50, cmap=cmap, vmin=vmin, vmax=vmax)

# Add shell boundaries
circle_R1 = plt.Circle(
    (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
    R1, color='red', fill=False, linestyle='--', linewidth=2, label='R1 (inner)'
)
circle_R2 = plt.Circle(
    (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
    R2, color='red', fill=False, linestyle=':', linewidth=2, label='R2 (outer)'
)
ax.add_patch(circle_R1)
ax.add_patch(circle_R2)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Energy Density $T^{00}$ (Eulerian Frame)', fontsize=14, fontweight='bold')
ax.set_aspect('equal')
ax.set_xlim(xlimit)
ax.set_ylim(xlimit)
ax.legend()

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Energy Density (J/m³)', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

print(f"\nEnergy Density Statistics:")
print(f"  Minimum: {np.min(energy_density[np.isfinite(energy_density)]):.3e} J/m³")
print(f"  Maximum: {np.max(energy_density[np.isfinite(energy_density)]):.3e} J/m³")
print(f"  Mean: {np.mean(energy_density[np.isfinite(energy_density)]):.3e} J/m³")

## Visualizing Momentum and Stress Components

The off-diagonal components of the stress-energy tensor represent momentum density and stress (shear) in the material.

In [ ]:
# Plot energy tensor components
energy_components = [
    (0, 1, 'X Momentum', True),
    (0, 2, 'Y Momentum', True),
    (1, 1, 'X Pressure', False),
    (1, 2, 'X-Y Shear', False),
    (2, 2, 'Y Pressure', False),
    (3, 3, 'Z Pressure', False)
]

fig = plt.figure(figsize=(18, 12))
fig.suptitle('Stress-Energy Tensor Components (Eulerian Frame)', fontsize=16, fontweight='bold')

for idx, (i, j, title, flip_sign) in enumerate(energy_components):
    ax = fig.add_subplot(2, 3, idx + 1)
    
    # Extract component
    data = results['energy_tensor_eulerian'][(i, j)][t_idx, 2:-2, 2:-2, z_idx].T
    
    # Flip sign for momentum components (convention)
    if flip_sign:
        data = -data
    
    # Get colormap
    cmap, vmin, vmax = get_diverging_colormap(data)
    
    # Plot
    im = ax.contourf(X, Y, data, levels=50, cmap=cmap, vmin=vmin, vmax=vmax)
    
    # Add shell boundaries
    circle_R1 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R1, color='red', fill=False, linestyle='--', linewidth=1
    )
    circle_R2 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R2, color='red', fill=False, linestyle=':', linewidth=1
    )
    ax.add_patch(circle_R1)
    ax.add_patch(circle_R2)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f'$T^{{{i}{j}}}$ - {title}')
    ax.set_aspect('equal')
    ax.set_xlim(xlimit)
    ax.set_ylim(xlimit)
    
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Energy Conditions

Energy conditions are inequalities that the stress-energy tensor must satisfy under certain physical assumptions:

1. **Null Energy Condition (NEC)**: $T_{\mu\nu} k^\mu k^\nu \geq 0$ for all null vectors $k^\mu$
2. **Weak Energy Condition (WEC)**: $T_{\mu\nu} t^\mu t^\nu \geq 0$ for all timelike vectors $t^\mu$
3. **Strong Energy Condition (SEC)**: $(T_{\mu\nu} - \frac{1}{2}T g_{\mu\nu}) t^\mu t^\nu \geq 0$ for all timelike vectors $t^\mu$
4. **Dominant Energy Condition (DEC)**: WEC + energy doesn't flow faster than light

Negative values indicate **violation** of the energy condition at that point.

In [ ]:
# Plot all energy conditions
energy_conditions = [
    ('null', 'Null Energy Condition (NEC)'),
    ('weak', 'Weak Energy Condition (WEC)'),
    ('strong', 'Strong Energy Condition (SEC)'),
    ('dominant', 'Dominant Energy Condition (DEC)')
]

fig = plt.figure(figsize=(16, 14))
fig.suptitle('Energy Conditions for Warp Shell', fontsize=16, fontweight='bold')

for idx, (condition_name, title) in enumerate(energy_conditions):
    ax = fig.add_subplot(2, 2, idx + 1)
    
    # Extract condition data
    data = results[condition_name][t_idx, 2:-2, 2:-2, z_idx].T
    
    # Get colormap
    cmap, vmin, vmax = get_diverging_colormap(data)
    
    # Plot
    im = ax.contourf(X, Y, data, levels=50, cmap=cmap, vmin=vmin, vmax=vmax)
    
    # Add shell boundaries
    circle_R1 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R1, color='red', fill=False, linestyle='--', linewidth=2
    )
    circle_R2 = plt.Circle(
        (world_center[1] / grid_scaling[1], world_center[2] / grid_scaling[2]),
        R2, color='red', fill=False, linestyle=':', linewidth=2
    )
    ax.add_patch(circle_R1)
    ax.add_patch(circle_R2)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.set_xlim(xlimit)
    ax.set_ylim(xlimit)
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Condition Value\n(negative = violation)', rotation=270, labelpad=25)

plt.tight_layout()
plt.show()

# Print statistics
print("\nEnergy Condition Violation Statistics:")
print("=" * 60)
for condition_name, title in energy_conditions:
    data = results[condition_name]
    valid_data = data[np.isfinite(data)]
    violations = valid_data[valid_data < 0]
    
    print(f"\n{title}:")
    print(f"  Violation points: {len(violations)}/{len(valid_data)} ({100*len(violations)/len(valid_data):.2f}%)")
    if len(violations) > 0:
        print(f"  Worst violation: {np.min(violations):.3e}")
        print(f"  Mean violation: {np.mean(violations):.3e}")
    print(f"  Maximum value: {np.max(valid_data):.3e}")

## Summary

In this notebook, we have:

1. **Created a warp shell metric** using `get_warp_shell_comoving_metric()`:
   - Defined shell geometry with inner radius R1 and outer radius R2
   - Computed density and pressure profiles using TOV equations
   - Applied smoothing to create realistic physical profiles
   - Enabled warp effect with velocity v_warp

2. **Examined the physical profiles**:
   - Density profile: constant within the shell, zero outside
   - Pressure profile: computed from TOV equations for hydrostatic equilibrium
   - Mass profile: integrated from density
   - Metric functions A and B: computed from Einstein field equations
   - Compactness parameter: measures how close to black hole formation

3. **Visualized the metric components**:
   - Showed spatial variation of $g_{\mu\nu}$ components
   - Observed how the metric deviates from flat spacetime in the shell region
   - Noted the $g_{01}$ component that enables the warp effect

4. **Computed and analyzed the stress-energy tensor**:
   - Energy density distribution
   - Momentum density components
   - Pressure and shear stress components

5. **Evaluated energy conditions**:
   - Null, Weak, Strong, and Dominant energy conditions
   - Identified regions where exotic matter is required
   - Quantified the severity of violations

### Key Physics Insights

- The **TOV equations** ensure the shell is in hydrostatic equilibrium
- The **warp effect** is created by the $g_{01}$ (time-space) metric component
- **Energy condition violations** indicate exotic matter requirements
- The **comoving frame** moves with the warp drive, simplifying the metric

### References

- Fell, S. D. B. & Heisenberg, L. (2024). "Constant velocity warp drive". *Classical and Quantum Gravity*, **41**(6). [https://iopscience.iop.org/article/10.1088/1361-6382/ad26aa](https://iopscience.iop.org/article/10.1088/1361-6382/ad26aa)

### Next Steps

Try modifying the parameters and observe the effects:
- Change `R1` and `R2` to vary shell thickness
- Adjust `v_warp` to change the warp velocity
- Modify `smooth_factor` to control profile smoothness
- Set `do_warp=False` to see the shell without warp effect
- Change `factor` to adjust the shell's mass/compactness